## Train MM / explore

In [1]:
import matplotlib

matplotlib.use("Agg")

import logging

logger = logging.getLogger()
logger.disabled = True

import os
from agent import DQNAgent
from tqdm.auto import tqdm
import random

for room_size in ["xxs-different-prob"]:
    params_all = []
    for capacity_max in [2]:
        for test_seed in [0]:
            semantic_decay_factor = 0.8
            pretrain_semantic = False
            num_iterations = 100 * 100
            batch_size = 2
            question_interval = 1
            terminates_at = 99

            capacity_h = {
                "episodic": capacity_max // 2,
                "semantic": capacity_max // 2,
                "short": 1,
            }
            capacity_e = {
                "episodic": capacity_max,
                "semantic": 0,
                "short": 1,
            }
            capacity_s = {
                "episodic": 0,
                "semantic": capacity_max,
                "short": 1,
            }
            if "different-prob" in room_size:
                root_path = f"./training-results/non-equal-object-probs/dqn/room_size={room_size}/capacity={capacity_max}/"
            else:
                root_path = f"./training-results/equal-object-probs/dqn/room_size={room_size}/capacity={capacity_max}/"

            for capacity in [capacity_h, capacity_e, capacity_s]:
                params = {
                    "env_str": "room_env:RoomEnv-v2",
                    "num_iterations": num_iterations,
                    "replay_buffer_size": num_iterations,
                    "warm_start": num_iterations // 10,
                    "batch_size": batch_size,
                    "target_update_interval": 10,
                    "epsilon_decay_until": num_iterations,
                    "max_epsilon": 1.0,
                    "min_epsilon": 0.1,
                    "gamma": {"mm": 0.99, "explore": 0.9},
                    "capacity": capacity,
                    "pretrain_semantic": pretrain_semantic,
                    "semantic_decay_factor": semantic_decay_factor,
                    "lstm_params": {
                        "num_layers": 2,
                        "embedding_dim": 64,
                        "hidden_size": 64,
                        "bidirectional": False,
                        "max_timesteps": terminates_at + 1,
                        "max_strength": terminates_at + 1,
                        "relu_for_attention": True,
                    },
                    "mlp_params": {
                        "hidden_size": 64,
                        "num_hidden_layers": 1,
                        "dueling_dqn": True,
                    },
                    "num_samples_for_results": {"val": 5, "test": 10},
                    "validation_interval": 5,
                    "plotting_interval": 50,
                    "train_seed": test_seed + 5,
                    "test_seed": test_seed,
                    "device": "cpu",
                    "qa_function": "episodic_semantic",
                    "explore_policy_heuristic": "avoid_walls",
                    "env_config": {
                        "question_prob": 1.0,
                        "terminates_at": terminates_at,
                        "randomize_observations": "objects",
                        "room_size": room_size,
                        "rewards": {"correct": 1, "wrong": 0, "partial": 0},
                        "make_everything_static": False,
                        "num_total_questions": 1000,
                        "question_interval": question_interval,
                        "include_walls_in_observations": True,
                    },
                    "ddqn": True,
                    "default_root_dir": f"./training-results/TRASH/",
                    # "default_root_dir": os.path.join(root_path, f"episodic={capacity['episodic']}_semantic={capacity['semantic']}/pretrain_semantic={pretrain_semantic}/")
                }
                params_all.append(params)

    for params in tqdm(params_all):
        agent = DQNAgent(**params)
        agent.train()

/home/tk/.virtualenvs/agent-room-env-v2-lstm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|          | 0/3 [00:00<?, ?it/s]

Running on cpu


/home/tk/.virtualenvs/agent-room-env-v2-lstm/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:168: DeprecationWarning: WARN: Current gymnasium version requires that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.deprecation(
/home/tk/.virtualenvs/agent-room-env-v2-lstm/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:181: DeprecationWarning: WARN: Current gymnasium version requires that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.deprecation(
/home/tk/.virtualenvs/agent-room-env-v2-lstm/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:127: UserWarning: WARN: The obs returned by the `reset()` method should be an int or np.int64, actual type: <class 'dict'>
  logger.warn(f"{pre} should be an int or np.int64, actual type: {type(obs)}")
/home/tk/.virtualenvs/agent-room-env-v2-ls

KeyboardInterrupt: 